In [0]:
!pip install python-twitter

     |████████████████████████████████| 71kB 4.3MB/s 


In [0]:
import twitter
import json

In [0]:
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving keys.py to keys.py


In [0]:
import keys

In [0]:
api = twitter.Api(consumer_key = keys.consumer_key,
                 consumer_secret = keys.consumer_secret,
                 access_token_key = keys.access_token,
                 access_token_secret = keys.access_token_secret)

In [0]:
import pandas as pd

In [0]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
  name = fn, length = len(uploaded[fn])))

Saving drug_safety_data.txt to drug_safety_data.txt
User uploaded file "drug_safety_data.txt" with length 185065 bytes


In [0]:
drugTweets = pd.read_csv('drug_safety_data.txt', delimiter = '\t', header = None, names = ['tweet_id', 'twitter_user_id', 'abuse'])

In [0]:
drugTweets = drugTweets.drop_duplicates()

In [0]:
drugTweets_text = api.GetStatuses(drugTweets.tweet_id)

In [0]:
txts = []
for tweet in drugTweets_text:
  txts.append(json.loads(json.dumps(tweet._json)))

In [0]:
txts[0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun May 12 18:08:41 +0000 2013',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 333644914913079296,
 'id_str': '333644914913079296',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'text': 'i know for a FACT that alcohol does not deplete the seroquel levels in your blood, YET HERE WE ARE',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Fri Dec 24 03:45:56 +0000 2010',
  'default_profile': False,
  'default_profile_image': False,
  'description': 'revive prime suspect (2011 - 2012)',
  'enti

In [0]:
ids = []
text = []
for line in txts:
  ids.append(line['id'])
  text.append(line['text'])

In [0]:
!pip install tweet-preprocessor

  Created wheel for tweet-preprocessor: filename=tweet_preprocessor-0.5.0-cp36-none-any.whl size=7946 sha256=512c6442e98ba74f728c53b6e4c4770ef992150474f86deae9e7ff6029dd5aa6
  Stored in directory: /root/.cache/pip/wheels/1b/27/cc/49938e98a2470802ebdefae9d2b3f524768e970c1ebbe2dc4a
Successfully built tweet-preprocessor


In [0]:
import preprocessor as p

In [0]:
text_clean = []
for line in text:
  text_clean.append(p.clean(line))

In [0]:
tweets_w_text = pd.DataFrame(list(zip(ids, text_clean, time)), columns = ['tweet_id', 'text_text', 'date'])

In [0]:
drugTweets_comb = pd.merge(tweets_w_text, drugTweets, on = 'tweet_id', how = 'inner')

In [0]:
drugTweets_comb.head(10)

,tweet_id,text_text,date,twitter_user_id,abuse
0,333644914913079296,i know for a FACT that alcohol does not deplet...,18,230052171,0
1,344825926342832128,mg Seroquel with my ‘normal’ mg Lovan and mcg ...,14,179074771,0
2,333854289023864832,SEVEN missed calls? get you're seroquel mg low...,08,333099736,0
3,344606561873833985,there's a fella on my Facebook who is asking t...,00,464202509,1
4,341947615853813761,you take vyvanse? I was on that stuff for like...,16,90304006,0
5,328655421256654849,antipsychotics: quetiapine (sedation); olanzap...,23,1371093139,0
6,333735685406224385,I take quetiapine and it's supposed to just re...,00,38919907,0
7,344937940146855937,Seroquel is pretty heavy stuff. I would've tho...,22,185070700,0
8,342112352130449409,Tell me why this kid just gave me six seroquel...,02,345065773,0
9,341680382179160065,look at the tweet near that one. I refuse to t...,22,38971420,0


In [0]:
abused_date = drugTweets_comb[drugTweets_comb.abuse == 1]
non_abused_date = drugTweets_comb[drugTweets_comb.abuse == 0]

In [0]:
non_abused_date.date.value_counts()

20    151
03    148
02    145
04    142
16    142
18    132
17    129
21    129
15    127
19    122
14    121
01    118
00    117
22    113
23    105
06     99
05     92
07     78
12     67
13     66
08     65
10     55
11     55
09     48
Name: date, dtype: int64

In [0]:
from google.colab import files
drugTweets_comb.to_csv('drugTweets_Case.txt', index=False, sep="\t", header=True) 
files.download('drugTweets_Case.txt')